In [29]:
## library imports here
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector, ColumnTransformer
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier
from sklearn.tree import DecisionTreeRegressor,DecisionTreeClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis, QuadraticDiscriminantAnalysis

import numpy as np

from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures,label_binarize
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import r2_score, mean_squared_error, confusion_matrix, accuracy_score, classification_report, roc_curve, auc,roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.metrics import make_scorer, f1_score
from sklearn.tree import DecisionTreeClassifier, plot_tree
from plotnine import ggplot, aes, geom_tile, labs, scale_fill_gradient, theme_minimal, element_text

# Classification
Target: Q4 - Liberal, Conservative, Moderate

In [30]:
cah_train = pd.read_csv('/Users/williamkapner/Documents/GSB_544/Data/CAH-201803-train.csv')
cah_test = pd.read_csv('/Users/williamkapner/Documents/GSB_544/Data/CAH-201803-test.csv')
cah_train.head()
#cah_test.head()

,id_num,Q1,Q2,political_affiliation,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18
0,1,Male,53,Independent,Liberal,College degree,Black,No,No,No,"Yes, somewhat religious",Pro-Choice,No,No,Behave no differently,5,2,5,No
1,5,Female,66,Independent,Conservative,Some college,White,Yes,No,Yes,"Yes, very religious",Pro-life,Yes,Yes,Less Willing,4,5,4,No
2,7,Female,58,Democrat,Liberal,College degree,White,No,No,No,"Yes, very religious",Pro-Choice,No,No,Behave no differently,5,1,4,Yes
3,8,Male,55,Independent,Moderate,High school or less,White,Yes,Yes,Yes,"Yes, somewhat religious",Pro-life,Yes,Yes,Less Willing,4,5,4,Yes
4,9,Male,64,Republican,Conservative,High school or less,White,Yes,Yes,Yes,No,Pro-life,No,No,Behave no differently,5,1,1,Yes


## LDA

In [58]:
# Essential imports
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report

# Assuming 'cah_train' is your DataFrame with the data
X = cah_train.drop(columns=['political_affiliation'])
y = cah_train['political_affiliation']

# Split data into train and test sets (you can adjust the test_size and random_state as needed)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Identify categorical and numeric columns
categorical_columns = X.select_dtypes(include=["object", "category"]).columns
numeric_columns = X.select_dtypes(include=["int64", "float64"]).columns

# Create a ColumnTransformer to apply different preprocessing steps
ct = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False), categorical_columns),
        ("standardize", StandardScaler(), numeric_columns)
    ],
    remainder="drop"  # Drop any columns not specified above
)

# Create a pipeline with preprocessing and LDA
pipeline = Pipeline([
    ('preprocessing', ct),  # Feature scaling
    ('lda', LinearDiscriminantAnalysis())  # LDA model
])

# Define parameter grid for GridSearchCV
param_grid = {
    'lda__solver': ['svd', 'lsqr', 'eigen'],  # Solvers to test
    'lda__shrinkage': [None, 'auto'],  # Shrinkage (only used with 'lsqr' and 'eigen')
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

# Fit GridSearchCV on training data
grid_search.fit(X_train, y_train)
grid_search.fit(X, y)


# Print the best parameters and best score from cross-validation
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated Accuracy:", grid_search.best_score_)

# Evaluate the final model on the test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Output confusion matrix and classification report for the test data
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))


Best Parameters: {'lda__shrinkage': 'auto', 'lda__solver': 'lsqr'}
Best Cross-Validated Accuracy: 0.615686274509804
Confusion Matrix:
 [[17  1  0]
 [ 3 11  3]
 [ 2  2 12]]
Classification Report:
               precision    recall  f1-score   support

    Democrat       0.77      0.94      0.85        18
 Independent       0.79      0.65      0.71        17
  Republican       0.80      0.75      0.77        16

    accuracy                           0.78        51
   macro avg       0.79      0.78      0.78        51
weighted avg       0.79      0.78      0.78        51



/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
10 fits failed out of a total of 30.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
3 fits failed with the following error:
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/base.py", line 1474, in wrapper
    return fit_method(estimator, *args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/pipeline.py", line 475, in fit
    self._final_estimator.

In [60]:
# Assuming best_model is the trained pipeline from GridSearchCV, which includes preprocessing

# Preprocess and make predictions on the new test data (cah_test)
# Drop the target column 'political_affiliation' if it exists in the new data
X_test_final = cah_test.drop(columns=['political_affiliation'], errors='ignore')

# Make predictions using the best model (which includes preprocessing)
political_affiliation_predicted = best_model.predict(X_test_final)

# Create a DataFrame to store the results
final_predictions = pd.DataFrame({
    "id_num": cah_test['id_num'],  # Assuming 'id_num' exists in cah_test
    "political_affiliation_predicted": political_affiliation_predicted
})

# Save predictions to a CSV file
final_predictions.to_csv("final_predictions_lda.csv", index=False)

# Optionally print the first few rows of the predictions to verify
print(final_predictions.head())


   id_num political_affiliation_predicted
0       2                      Republican
1       3                        Democrat
2       4                     Independent
3       6                     Independent
4      11                     Independent


## QDA

In [57]:
pipeline = Pipeline([
    ('preprocessing', ct),  # Feature scaling
    ('qda', QuadraticDiscriminantAnalysis())  # QDA model
])

# Define parameter grid
param_grid = {
    'qda__reg_param': [0.0, 0.1, 0.2, 0.5, 1.0],  # Regularization parameters
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

# Fit GridSearchCV on training data
grid_search.fit(X_train, y_train)
grid_search.fit(X, y)


# Print the best parameters and best score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated Accuracy:", grid_search.best_score_)

# Evaluate the final model on test data
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Output confusion matrix and classification report
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Best Parameters: {'qda__reg_param': 0.5}
Best Cross-Validated Accuracy: 0.5859180035650624
Confusion Matrix:
 [[15  1  2]
 [ 3 12  2]
 [ 2  2 12]]
Classification Report:
               precision    recall  f1-score   support

    Democrat       0.75      0.83      0.79        18
 Independent       0.80      0.71      0.75        17
  Republican       0.75      0.75      0.75        16

    accuracy                           0.76        51
   macro avg       0.77      0.76      0.76        51
weighted avg       0.77      0.76      0.76        51



/opt/anaconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.12/site-packages/sklearn/discriminant_analysis.py:935: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.1

## Best Prediction

In [59]:
cah_test.head()

,id_num,Q1,Q2,Q4,Q5,Q6,Q7,Q8,Q9,Q10,Q11,Q12,Q13,Q14,Q15,Q16,Q17,Q18
0,2,Female,78,Conservative,College degree,White,Yes,Yes,No,"Yes, very religious",Pro-Choice,Yes,Yes,Behave no differently,4,5,1,Yes
1,3,Male,59,Moderate,High school or less,Black,Yes,Yes,Yes,"Yes, very religious",Pro-Choice,No,No,More Willing,5,4,5,No
2,4,Male,59,Moderate,High school or less,White,Yes,No,Yes,"Yes, very religious",Pro-life,Yes,No,Behave no differently,4,5,1,Yes
3,6,Male,52,Moderate,Graduate degree,White,Yes,Yes,Yes,"Yes, somewhat religious",Pro-Choice,No,Yes,Less Willing,5,4,4,No
4,11,Female,33,Moderate,High school or less,White,No,No,Yes,"Yes, somewhat religious",Pro-Choice,No,No,More Willing,5,5,4,Yes


In [39]:
final_predictions = pd.DataFrame(
    {"id_num": cah_test['id_num'],
    "political_affiliation_predicted": best_model.predict(cah_test)}
)
final_predictions

,id_num,political_affiliation_predicted
0,2,Republican
1,3,Independent
2,4,Republican
3,6,Independent
4,11,Independent
...,...,...
161,327,Democrat
162,330,Independent
163,331,Republican
164,333,Independent


In [61]:
final_predictions.to_csv("final_predictions.csv", index=False)

## SVC

In [56]:
from sklearn.svm import LinearSVC
pipeline = Pipeline([
    ('preprocessing', ct),  # Feature scaling
    ('linsvc', LinearSVC())  # Linear Support Vector Classifier
])

# Define parameter grid for hyperparameter tuning
param_grid = {
    'linsvc__C': [0.1, 1, 10],  # Regularization parameter
    'linsvc__max_iter': [1000, 5000],  # Maximum number of iterations
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.fit(X, y)


# Output the best parameters and best cross-validated score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated Accuracy:", grid_search.best_score_)

# Evaluate the final model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Output confusion matrix and classification report
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/

Best Parameters: {'linsvc__C': 0.1, 'linsvc__max_iter': 1000}
Best Cross-Validated Accuracy: 0.6037433155080214
Confusion Matrix:
 [[16  1  1]
 [ 3 11  3]
 [ 2  2 12]]
Classification Report:
               precision    recall  f1-score   support

    Democrat       0.76      0.89      0.82        18
 Independent       0.79      0.65      0.71        17
  Republican       0.75      0.75      0.75        16

    accuracy                           0.76        51
   macro avg       0.77      0.76      0.76        51
weighted avg       0.77      0.76      0.76        51



/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_base.py:1237: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(
/opt/anaconda3/lib/python3.12/site-packages/sklearn/svm/_classes.py:31: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.


## SVM

In [55]:
#SVM
from sklearn.svm import SVC

pipeline = Pipeline([
    ('preprocessing', ct),  # Feature scaling
    ('svc', SVC())  # Support Vector Classifier
])

# Define parameter grid for hyperparameter tuning
param_grid = {
    'svc__kernel': ['poly'],  # Polynomial kernel
    'svc__degree': [2, 3, 4],  # Degree of polynomial
    'svc__C': [0.1, 1, 10],  # Regularization parameter
    'svc__gamma': ['scale', 'auto']  # Kernel coefficient
}

# Initialize GridSearchCV
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', verbose=0, n_jobs=-1)

# Fit the grid search to the data
grid_search.fit(X_train, y_train)
grid_search.fit(X, y)

# Output the best parameters and best cross-validated score
print("Best Parameters:", grid_search.best_params_)
print("Best Cross-Validated Accuracy:", grid_search.best_score_)

# Evaluate the final model on the test set
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)

# Output confusion matrix and classification report
print("Confusion Matrix:\n", confusion_matrix(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

Best Parameters: {'svc__C': 1, 'svc__degree': 2, 'svc__gamma': 'auto', 'svc__kernel': 'poly'}
Best Cross-Validated Accuracy: 0.5798573975044563
Confusion Matrix:
 [[17  0  1]
 [ 4 10  3]
 [ 5  1 10]]
Classification Report:
               precision    recall  f1-score   support

    Democrat       0.65      0.94      0.77        18
 Independent       0.91      0.59      0.71        17
  Republican       0.71      0.62      0.67        16

    accuracy                           0.73        51
   macro avg       0.76      0.72      0.72        51
weighted avg       0.76      0.73      0.72        51



In [53]:
# Assuming best_model is the trained pipeline from GridSearchCV, which includes preprocessing

# Preprocess and make predictions on the new test data (cah_test)
# Drop the target column 'political_affiliation' if it exists in the new data
X_test_final = cah_test.drop(columns=['political_affiliation'], errors='ignore')

# Make predictions using the best model (which includes preprocessing)
political_affiliation_predicted = best_model.predict(X_test_final)

# Create a DataFrame to store the results
final_predictions = pd.DataFrame({
    "id_num": cah_test['id_num'],  # Assuming 'id_num' exists in cah_test
    "political_affiliation_predicted": political_affiliation_predicted
})

# Save predictions to a CSV file
final_predictions.to_csv("final_predictions_svm2.csv", index=False)

# Optionally print the first few rows of the predictions to verify
print(final_predictions.head())


   id_num political_affiliation_predicted
0       2                      Republican
1       3                     Independent
2       4                      Republican
3       6                     Independent
4      11                     Independent


## Logistic Regression

In [45]:
# Logistic regression pipeline
logreg_pipeline = Pipeline([
    ('preprocessing',ct),
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))
])

# Fit the model on training data
logreg_model = logreg_pipeline.fit(X, y)



cv_scores_accuracy = cross_val_score(logreg_pipeline, X, y, cv=5, scoring='accuracy')

# Print cross-validated accuracy
print("Cross-Validated Accuracy Scores for Each Fold:", cv_scores_accuracy)
print("Mean Cross-Validated Accuracy:", cv_scores_accuracy.mean())

# Fit the final model and make predictions on the training set
final_logreg_model = logreg_pipeline.fit(X, y)
y_train_pred_log = final_logreg_model.predict(X)

# Compute and print the confusion matrix
conf_matrix = confusion_matrix(y, y_train_pred_log)
print("\nConfusion Matrix:\n", conf_matrix)

Cross-Validated Accuracy Scores for Each Fold: [0.41176471 0.58823529 0.70588235 0.67647059 0.57575758]
Mean Cross-Validated Accuracy: 0.5916221033868093

Confusion Matrix:
 [[44 13  2]
 [11 40  5]
 [ 5  5 44]]


In [49]:
# Essential imports
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.multiclass import OneVsOneClassifier, OneVsRestClassifier

# Check if your categorical columns have non-numeric values
categorical_columns = X.select_dtypes(include=["object", "category"]).columns
numeric_columns = X.select_dtypes(include=["int64", "float64"]).columns

# Handle missing values (if applicable), this is a simple approach with 'fillna'
# Alternatively, you can use more sophisticated imputation methods

# Create a ColumnTransformer to preprocess categorical and numerical columns
ct = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(handle_unknown='ignore', sparse_output=False), categorical_columns),  # One-hot encoding for categorical columns
        ("standardize", StandardScaler(), numeric_columns)  # Standardize numeric columns
    ],
    remainder="drop"  # Drop any columns that are not specified above
)

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# Create base pipeline (preprocessing + logistic regression)
base_pipeline = Pipeline([
    ('preprocessing', ct),  # Preprocessing step
    ('logreg', LogisticRegression(max_iter=1000, random_state=42))  # Logistic Regression model
])

# One-vs-One (OvO) model
ovo_model = OneVsOneClassifier(base_pipeline)
ovo_model.fit(X_train, y_train)

# Cross-validation for OvO model
cv_scores_ovo = cross_val_score(ovo_model, X_train, y_train, cv=5, scoring='accuracy')
print("OvO Cross-Validated Accuracy Scores:", cv_scores_ovo)
print("Mean OvO Cross-Validated Accuracy:", cv_scores_ovo.mean())

# One-vs-Rest (OvR) model
ovr_model = OneVsRestClassifier(base_pipeline)
ovr_model.fit(X_train, y_train)

# Cross-validation for OvR model
cv_scores_ovr = cross_val_score(ovr_model, X_train, y_train, cv=5, scoring='accuracy')
print("OvR Cross-Validated Accuracy Scores:", cv_scores_ovr)
print("Mean OvR Cross-Validated Accuracy:", cv_scores_ovr.mean())

# Predict on test data for confusion matrices
y_test_pred_ovo = ovo_model.predict(X_test)
y_test_pred_ovr = ovr_model.predict(X_test)

# Confusion matrices for test set predictions
conf_matrix_ovo = confusion_matrix(y_test, y_test_pred_ovo)
conf_matrix_ovr = confusion_matrix(y_test, y_test_pred_ovr)

print("\nConfusion Matrix for OvO on Test Data:\n", conf_matrix_ovo)
print("\nConfusion Matrix for OvR on Test Data:\n", conf_matrix_ovr)


ValueError: could not convert string to float: 'Female'

In [66]:
X = cah_test
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

cah_test = pd.get_dummies(cah_test)
cah_test_predictions = best_model.predict(cah_test)

final_predictions = pd.DataFrame(
    {"id_num": cah_test['id_num'],
    "political_affiliation_predicted": ovo_model.predict(cah_test)}
)
final_predictions
final_predictions.to_csv("final_predictions_OvO.csv", index=False)

# Regression
Target: SalePrice

In [101]:
housing_train = pd.read_csv('/Users/williamkapner/Documents/GSB_544/Data/train_new.csv')
housing_test = pd.read_csv('/Users/williamkapner/Documents/GSB_544/Data/test_new.csv')
housing_train.dropna(inplace=True)
housing_train.head()
housing_test.head()

,PID,Lot Frontage,Lot Area,Street,Neighborhood,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,...,Full Bath,Half Bath,Bedroom AbvGr,TotRms AbvGrd,Gr Liv Area,Functional,Screen Porch,Pool Area,Yr Sold,Sale Type
0,907135180,60,8070,Pave,CollgCr,1Fam,1Story,4,5,1994,...,1,0,3,5,990,Typ,0,0,2007,WD
1,528181040,40,6792,Pave,NridgHt,TwnhsE,1Story,7,5,2005,...,2,0,2,6,1368,Typ,0,0,2006,New
2,528175010,44,6371,Pave,NridgHt,TwnhsE,1Story,7,5,2009,...,2,0,2,6,1358,Typ,0,0,2010,New
3,531379030,70,8304,Pave,SawyerW,1Fam,2Story,6,5,1997,...,2,1,3,7,1837,Typ,0,0,2006,WD
4,923275090,37,6951,Pave,Mitchel,1Fam,1Story,5,5,1984,...,1,0,3,5,923,Typ,0,0,2008,WD


In [106]:
from sklearn.preprocessing import LabelEncoder
housing_train = housing_train.dropna(subset=['SalePrice'])
y = housing_train['SalePrice']
# X = housing_train.drop(columns=['SalePrice'])
# y = housing_train['SalePrice']


categorical_columns = [col for col in X.select_dtypes(include=["object", "category"]).columns if col in X.columns]
numeric_columns = [col for col in X.select_dtypes(include=["int64", "float64"]).columns if col in X.columns]

# Create a ColumnTransformer
ct = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output=False, handle_unknown="ignore"), categorical_columns),
        ("standardize", StandardScaler(), numeric_columns),
    ],
    remainder="drop"  # Drop any columns not specified
) 

y

0       159000
1       271900
3       248500
4       167000
5       140000
         ...  
2192    220000
2193    160000
2194    225000
2195     83000
2196    250000
Name: SalePrice, Length: 1834, dtype: int64

## Linear, Random Forest

In [103]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.metrics import make_scorer, mean_squared_error

# Define log-transformed RMSE scorer
def log_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

log_rmse_scorer = make_scorer(log_rmse, greater_is_better=False)

# Preprocess the data
X = housing_train.drop(columns=['SalePrice', 'PID'])
y = housing_train['SalePrice']

categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ])

# Define models to compare
models = {
    'Linear Regression': LinearRegression(),
    'Random Forest': RandomForestRegressor(n_estimators=100, random_state=42),
}

# Cross-validation for each model
results = {}
for model_name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X, y, cv=5, scoring=log_rmse_scorer)
    mean_rmse = -scores.mean()  # Negate to convert to positive RMSE
    results[model_name] = mean_rmse
    print(f"{model_name}: Mean Log-RMSE = {mean_rmse:.5f}")

# Select the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name} with Log-RMSE = {results[best_model_name]:.5f}")

# Train the best model on the full dataset
best_model = models[best_model_name]
best_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_model)
])
best_pipeline.fit(X, y)

# Predict on test data
test_data_preprocessed = housing_test.drop(columns=['PID'])
test_predictions = np.expm1(best_pipeline.predict(test_data_preprocessed))  # Exponentiate predictions

# Save the submission
submission = pd.DataFrame({'PID': housing_test['PID'], 'SalePrice': test_predictions})
submission.to_csv('housing_submission_RF.csv', index=False)


Linear Regression: Mean Log-RMSE = 0.19221
Random Forest: Mean Log-RMSE = 0.15120
Best Model: Random Forest with Log-RMSE = 0.15120


/var/folders/ss/ytdznf2d6mxcx8d7tp5l1qb40000gn/T/ipykernel_1434/3457129724.py:69: RuntimeWarning: overflow encountered in expm1


## Linear, Ridge, Lasso, Elastic Net, Logistic

In [104]:
import pandas as pd
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.linear_model import (
    LinearRegression, Ridge, Lasso, ElasticNet, LogisticRegression
)
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error, make_scorer

# Define custom log-transformed RMSE scoring function
def log_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(np.log1p(y_true), np.log1p(y_pred)))

log_rmse_scorer = make_scorer(log_rmse, greater_is_better=False)

# Load and preprocess data
X = housing_train.drop(columns=['SalePrice', 'PID'])
y = housing_train['SalePrice']
categorical_cols = X.select_dtypes(include=['object']).columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', Pipeline([
            ('imputer', SimpleImputer(strategy='median')),
            ('scaler', StandardScaler())
        ]), numeric_cols),
        ('cat', Pipeline([
            ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
            ('onehot', OneHotEncoder(handle_unknown='ignore'))
        ]), categorical_cols)
    ])

# Define models to evaluate
models = {
    "Linear Regression": LinearRegression(),
    "Ridge Regression": Ridge(alpha=1.0),
    "LASSO Regression": Lasso(alpha=0.01),
    "Elastic Net": ElasticNet(alpha=0.01, l1_ratio=0.5),
    # Logistic Regression for demonstration purposes (binary target, optional)
}

# Evaluate each model
results = {}
for model_name, model in models.items():
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    scores = cross_val_score(pipeline, X, y, cv=5, scoring=log_rmse_scorer)
    mean_rmse = -scores.mean()  # Convert negative scores back to RMSE
    results[model_name] = mean_rmse
    print(f"{model_name}: Mean Log-RMSE = {mean_rmse:.5f}")

# Print the best model
best_model_name = min(results, key=results.get)
print(f"Best Model: {best_model_name} with Log-RMSE = {results[best_model_name]:.5f}")


Linear Regression: Mean Log-RMSE = 0.19221
Ridge Regression: Mean Log-RMSE = 0.19024


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 289623838684.6735, tolerance: 976072253.4102794
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 147747785353.43542, tolerance: 1080314885.2177987
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 9491068066.086914, tolerance: 1082993607.1659615
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 139638796315.34265, toler

LASSO Regression: Mean Log-RMSE = 0.19384
Elastic Net: Mean Log-RMSE = nan
Best Model: Ridge Regression with Log-RMSE = 0.19024


/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 25906100452.709717, tolerance: 1049866246.8647466
/var/folders/ss/ytdznf2d6mxcx8d7tp5l1qb40000gn/T/ipykernel_1434/1123123895.py:15: RuntimeWarning: invalid value encountered in log1p
/opt/anaconda3/lib/python3.12/site-packages/sklearn/model_selection/_validation.py:1011: UserWarning: Scoring failed. The score on this train-test partition for these parameters will be set to nan. Details: 
Traceback (most recent call last):
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 137, in __call__
    score = scorer._score(
            ^^^^^^^^^^^^^^
  File "/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_scorer.py", line 350, in _score
    return self._sign * self._score_func(y_true, y_pred, **scoring_kwargs)
                        ^^^^^^^^^^

## Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNetCV

# Elastic Net with cross-validation for parameter tuning
elastic_net = ElasticNetCV(
    alphas=np.logspace(-3, 1, 10),
    l1_ratio=np.linspace(0.1, 0.9, 5), 
    cv=5
)
pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('elastic_net', elastic_net)
])

scores = cross_val_score(pipeline, X, y, cv=5, scoring=log_rmse_scorer)
print(f"Elastic Net Mean Log RMSE: {-scores.mean():.5f}")

/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 123164127985.08374, tolerance: 766725826.2907286
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 91791417806.56726, tolerance: 805520052.8736277
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 72984237834.12756, tolerance: 802325304.49305
/opt/anaconda3/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:639: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 172639792556.62354, tolerance

Elastic Net Mean Log RMSE: 0.19404


## Tuning:

In [132]:
import numpy as np
from sklearn.linear_model import Ridge, Lasso, ElasticNet
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd

# Define custom RMSE scoring function
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Preprocessor for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Define preprocessors
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Log-transform the target variable
y_log = np.log(y)

# Define models
models = {
    "Ridge Regression": Ridge(random_state=42),
    "LASSO Regression": Lasso(random_state=42, max_iter=10000),
    "Elastic Net": ElasticNet(random_state=42, max_iter=10000)
}

# Define hyperparameter grids for each model
param_grids = {
    "Ridge Regression": {
        "model__alpha": [0.01, 0.1, 1.0, 10.0, 100.0]
    },
    "LASSO Regression": {
        "model__alpha": [0.001, 0.01, 0.1, 1.0, 10.0]
    },
    "Elastic Net": {
        "model__alpha": [0.001, 0.01, 0.1, 1.0],
        "model__l1_ratio": [0.1, 0.5, 0.9]
    },
}

# Perform grid search for each model
tuned_results = {}
for model_name, param_grid in param_grids.items():
    model = models[model_name]
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('model', model)
    ])
    grid_search = GridSearchCV(
        pipeline, param_grid, cv=5, scoring=rmse_scorer, n_jobs=-1
    )
    grid_search.fit(X, y_log)  # Fit using log-transformed target
    best_rmse = -grid_search.best_score_  # Convert negative RMSE to positive
    tuned_results[model_name] = (best_rmse, grid_search.best_params_)
    print(f"{model_name}: Best Log-RMSE = {best_rmse:.5f}, Best Params = {grid_search.best_params_}")

# Identify the overall best model
best_model_name = min(tuned_results, key=lambda x: tuned_results[x][0])
best_rmse, best_params = tuned_results[best_model_name]
print(f"Overall Best Model: {best_model_name} with Log-RMSE = {best_rmse:.5f}")
print(f"Best Parameters: {best_params}")

# Refit the best model on the entire dataset
best_model = models[best_model_name].set_params(**{key.split('__')[1]: val for key, val in best_params.items()})
best_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', best_model)
])
best_pipeline.fit(X, y_log)

# Make predictions on test set
X_test_final = housing_test.drop(columns=['political_affiliation'], errors='ignore')
log_predictions = best_pipeline.predict(X_test_final)

# Convert predictions back to original scale
sale_price_predicted = np.exp(log_predictions)

# Create a DataFrame for predictions
final_predictions = pd.DataFrame({
    "PID": housing_test['PID'],  # Assuming 'PID' exists in the test set
    "SalePrice": sale_price_predicted
})

# Save predictions to a CSV file
final_predictions.to_csv("housing_submission_EN.csv", index=False)

# Optionally print the first few rows of predictions
print(final_predictions.head())


Ridge Regression: Best Log-RMSE = 0.15308, Best Params = {'model__alpha': 10.0}
LASSO Regression: Best Log-RMSE = 0.15591, Best Params = {'model__alpha': 0.001}
Elastic Net: Best Log-RMSE = 0.15273, Best Params = {'model__alpha': 0.001, 'model__l1_ratio': 0.1}
Overall Best Model: Elastic Net with Log-RMSE = 0.15273
Best Parameters: {'model__alpha': 0.001, 'model__l1_ratio': 0.1}
         PID      SalePrice
0  907135180  127475.820154
1  528181040  220408.130200
2  528175010  219725.969779
3  531379030  185056.979983
4  923275090  129218.919321


## KNN

In [107]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

# Define the pipeline for KNN
knn_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),  # Preprocessing pipeline
    ('model', KNeighborsRegressor())  # KNN regressor
])

# Define the hyperparameter grid for KNN
knn_param_grid = {
    "model__n_neighbors": [3, 5, 10, 20],  # Number of neighbors
    "model__weights": ["uniform", "distance"],  # Weighting scheme
    "model__metric": ["euclidean", "manhattan"]  # Distance metric
}

# Perform GridSearchCV for KNN
knn_grid_search = GridSearchCV(
    knn_pipeline,
    knn_param_grid,
    cv=5,  # 5-fold cross-validation
    scoring=log_rmse_scorer,  # Custom log RMSE scorer
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV to the data
knn_grid_search.fit(X, y)

# Extract the best RMSE and parameters
best_knn_rmse = -knn_grid_search.best_score_  # Convert negative log RMSE to positive
best_knn_params = knn_grid_search.best_params_

print(f"KNN Regression: Best Log-RMSE = {best_knn_rmse:.5f}")
print(f"KNN Regression: Best Parameters = {best_knn_params}")


KNN Regression: Best Log-RMSE = 0.17147
KNN Regression: Best Parameters = {'model__metric': 'manhattan', 'model__n_neighbors': 5, 'model__weights': 'distance'}


## Decision Tree

In [108]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV

# Define the pipeline for Decision Tree
tree_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DecisionTreeRegressor(random_state=42))
])

# Define the hyperparameter grid for Decision Tree
tree_param_grid = {
    "model__max_depth": [5, 10, 20, None],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4]
}

# Perform GridSearchCV for Decision Tree
tree_grid_search = GridSearchCV(
    tree_pipeline,
    tree_param_grid,
    cv=5,  # 5-fold cross-validation
    scoring=log_rmse_scorer,  # Custom log RMSE scorer
    n_jobs=-1  # Use all available cores
)

# Fit GridSearchCV to the data
tree_grid_search.fit(X, y)

# Extract the best RMSE and parameters
best_tree_rmse = -tree_grid_search.best_score_  # Convert negative log RMSE to positive
best_tree_params = tree_grid_search.best_params_

print(f"Decision Tree Regression: Best Log-RMSE = {best_tree_rmse:.5f}")
print(f"Decision Tree Regression: Best Parameters = {best_tree_params}")


Decision Tree Regression: Best Log-RMSE = 0.18186
Decision Tree Regression: Best Parameters = {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 10}


## Random Forest Regressor

In [109]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

rf_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', RandomForestRegressor(random_state=42))
])

rf_param_grid = {
    "model__n_estimators": [100, 200, 300],
    "model__max_depth": [10, 20, None],
    "model__min_samples_split": [2, 5, 10],
    "model__min_samples_leaf": [1, 2, 4]
}

rf_grid_search = GridSearchCV(
    rf_pipeline,
    rf_param_grid,
    cv=5,
    scoring=log_rmse_scorer,
    n_jobs=-1
)

rf_grid_search.fit(X, y)

best_rf_rmse = -rf_grid_search.best_score_
best_rf_params = rf_grid_search.best_params_

print(f"Random Forest Regression: Best Log-RMSE = {best_rf_rmse:.5f}")
print(f"Random Forest Regression: Best Parameters = {best_rf_params}")


Random Forest Regression: Best Log-RMSE = 0.15053
Random Forest Regression: Best Parameters = {'model__max_depth': None, 'model__min_samples_leaf': 1, 'model__min_samples_split': 2, 'model__n_estimators': 300}


In [113]:
X_test_final = housing_test.drop(columns=['political_affiliation'], errors='ignore')

# Ensure any preprocessing steps are included in the model pipeline
best_rf_model = rf_grid_search.best_estimator_

# Make predictions using the best model (including preprocessing)
sale_price_predicted = best_rf_model.predict(X_test_final)

# Create a DataFrame to store the results
final_predictions = pd.DataFrame({
    "PID": housing_test['PID'],  # Assuming 'id_num' exists in cah_test
    "SalePrice": sale_price_predicted  # Replace with actual target variable name
})

# Save predictions to a CSV file
final_predictions.to_csv("housing_submission_rf1.csv", index=False)

# Optionally print the first few rows of the predictions to verify
print(final_predictions.head())

         PID      SalePrice
0  907135180  139394.666667
1  528181040  199380.496667
2  528175010  215716.263333
3  531379030  203197.166667
4  923275090  129145.413333


## Polynomial

In [128]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer, mean_squared_error
import pandas as pd
import numpy as np

# Define a custom RMSE scoring function
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

rmse_scorer = make_scorer(rmse, greater_is_better=False)

# Separate numeric and categorical columns
numeric_cols = X.select_dtypes(include=['int64', 'float64']).columns
categorical_cols = X.select_dtypes(include=['object']).columns

# Preprocessing pipelines for numeric and categorical data
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='mean')),  # Impute missing values with mean
    ('scaler', StandardScaler())  # Standardize numeric features
])

categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='Missing')),  # Impute missing values
    ('onehot', OneHotEncoder(handle_unknown='ignore'))  # One-hot encode categorical data
])

# Combine preprocessing steps
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_cols),
        ('cat', categorical_transformer, categorical_cols)
    ]
)

# Define the pipeline for polynomial regression
poly_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('polynomial_features', PolynomialFeatures(degree=2, include_bias=False)),
    ('linear_regression', Ridge())  # Ridge regression to handle overfitting
])

# Define the hyperparameter grid for tuning
param_grid = {
    'polynomial_features__degree': [2, 3, 4],  # Test different polynomial degrees
    'linear_regression__alpha': [0.01, 0.1, 1, 10, 100]  # Regularization strength for Ridge
}

# Set up GridSearchCV for cross-validation and hyperparameter tuning
grid_search = GridSearchCV(
    poly_pipeline,
    param_grid=param_grid,
    scoring=rmse_scorer,
    cv=5,
    n_jobs=-1
)

# Fit the model using GridSearchCV
grid_search.fit(X, y)

# Extract the best model and results
best_rmse = -grid_search.best_score_  # Convert negative score back to positive RMSE
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_

# Output the best results
print(f"Best Polynomial Regression Log-RMSE: {best_rmse:.5f}")
print(f"Best Parameters: {best_params}")

# Make predictions on the training set (or replace with test set)
y_pred = best_model.predict(X)

# Example of saving predictions to a CSV (if applicable)
predictions = pd.DataFrame({
    'Actual': y,
    'Predicted': y_pred
})

predictions.to_csv("quadratic_regression_predictions.csv", index=False)

# Optionally, print the first few rows of predictions
print(predictions.head())


KeyboardInterrupt: 

## XG Boost Tree Model

https://xgboost.readthedocs.io/en/stable/tutorials/model.html
Chat GPT taught me about this boosted tree model, and the above documentation helped as well. This tree model is just like a decision tree with a regularization parameter.

In [129]:
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score

# Add XGBoost to the pipeline
xgb_pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', XGBRegressor(objective='reg:squarederror', random_state=42))
])

# Define parameter grid for XGBoost
xgb_param_grid = {
    "model__n_estimators": [100, 500, 1000],
    "model__learning_rate": [0.01, 0.1, 0.3],
    "model__max_depth": [3, 5, 7],
    "model__subsample": [0.8, 1.0]
}

# Perform GridSearchCV for XGBoost
xgb_grid_search = GridSearchCV(
    xgb_pipeline, xgb_param_grid, cv=5, scoring=log_rmse_scorer, n_jobs=-1
)
xgb_grid_search.fit(X, y)

# Best RMSE and parameters
best_xgb_rmse = -xgb_grid_search.best_score_
best_xgb_params = xgb_grid_search.best_params_
best_xgb_model = xgb_grid_search.best_estimator_


print(f"XGBoost: Best Log-RMSE = {best_xgb_rmse:.5f}")
print(f"XGBoost: Best Parameters = {best_xgb_params}")


XGBoost: Best Log-RMSE = 0.13986
XGBoost: Best Parameters = {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 500, 'model__subsample': 0.8}


In [130]:
X_test_final = housing_test.drop(columns=['political_affiliation'], errors='ignore')

# Make predictions using the best XGBoost model
sale_price_predicted = best_xgb_model.predict(X_test_final)

# Create a DataFrame to store the results
final_predictions = pd.DataFrame({
    "PID": housing_test['PID'],  # Assuming 'PID' is the unique identifier in your test set
    "SalePrice": sale_price_predicted  # Predicted target variable
})

# Save predictions to a CSV file
final_predictions.to_csv("housing_submission.csv", index=False)

# Optionally print the first few rows of predictions
print(final_predictions.head())

         PID      SalePrice
0  907135180  135181.156250
1  528181040  210901.140625
2  528175010  208027.671875
3  531379030  199732.765625
4  923275090  121306.046875


Chat GPT-4o was used to automate the creation of the OvO and OvR models. It also taught me how to implement the XGBoost model with pipelines. It also automated some model creation.